In [1]:
from bs4 import BeautifulSoup
import os
import json
import utils
import pandas as pd
import time
import random
from datetime import date

In [2]:
def get_file_name(is_eng = False):
    today = date.today()
    datestr = today.strftime("%d_%m")
    file_name = "news_data_" + datestr +".csv"
    return file_name

In [3]:
file_name = get_file_name()

In [4]:
if not os.path.exists(os.path.join(".", file_name)):
    news_data = pd.DataFrame(data = {"link": []})
    news_data.to_csv(os.path.join(".", file_name))
else:
    news_data = pd.read_csv(os.path.join(".", file_name), index_col=0)

In [5]:
news_data.head()
news_data.shape

(9718, 1)

In [6]:
list_webs = json.load(open(os.path.join(".", "list_webs.json"), encoding='UTF-8'))

In [7]:
print(news_data.shape)
for web_obj in list_webs:
    if not web_obj.get("status", True) and len(web_obj.get("path")) > 0:
        if web_obj.get("type_crawl", -1) == 1:
            page = web_obj.get("info").get("start_from_page", 1) - 1
            while True:
                
                time.sleep(random.randint(20,30)/10)
                
                page += 1
                list_link = []
                if page == -1:
                    link = web_obj.get("path")
                else:
                    link = web_obj.get("path") + web_obj.get("info").get("prefix") + str(page) + web_obj.get("info").get("postfix")
                # print(link)
                list_link = utils.crawl_list_news(link = link, 
                                                  tag = web_obj.get("info").get("tag", None), 
                                                  attr = web_obj.get("info").get("attr", None),
                                                  domain = web_obj.get("info").get("result_domain", ""),
                                                  result_end = web_obj.get("info").get("result_end", ""))
                if len(list_link) == 0:
                    break
                else:
                    # print(list_link)
                    # break
                    news_data = pd.concat([news_data, pd.DataFrame({"link": list_link})], ignore_index=True, axis=0)
                    news_data = news_data.drop_duplicates(ignore_index=True)
                    news_data.to_csv(os.path.join(".", file_name))
                if page == -1 or page == web_obj.get("info").get("stop_at_page", -2):
                    break
            # break
            web_obj["status"] = True
            with open(os.path.join(".", "list_webs.json"), "w") as f:
                f.write(json.dumps(list_webs, indent = 4))
            # break
        elif web_obj.get("type_crawl", -1) == 2:
            page = web_obj.get("info").get("start_from_page", 1) - 1
            while True:
                
                time.sleep(random.randint(50,60)/10)
                page += 1
                
                list_link = utils.crawl_list_news_gg(domain=web_obj.get("path"),
                                                     page=page,
                                                     result_end= web_obj.get("info").get("result_end", ""))
                if len(list_link) == 0:
                    break
                else:
                    # print(list_link)
                    # break
                    news_data = pd.concat([news_data, pd.DataFrame({"link": list_link})], ignore_index=True, axis=0)
                    news_data = news_data.drop_duplicates(ignore_index=True)
                    news_data.to_csv(os.path.join(".", file_name))
                if page == -1 or page == web_obj.get("info").get("stop_at_page", -2):
                    break
            # break
            web_obj["status"] = True
            with open(os.path.join(".", "list_webs.json"), "w") as f:
                f.write(json.dumps(list_webs, indent = 4))
            # break
        print(news_data.shape)
        time.sleep(300)

(9718, 1)
(9968, 1)
(10215, 1)
(10231, 1)
